In [1]:
specs = {'adaptingDataframe': {'changeType': {}, 'renameMap': {}, 'setValue': None, 'colName': 'id_indicateur',
                               'value': '??', 'toDrop': []}, 'catalogUri': 'None',
         'confDb': {'chunksize': 1000, 'schema': 'surfor', 'strategy': 'append',
                    'tableName': 'faits_GFC_treecover2000'},
         'confDims': {'isin_id_mesure': [3, 100], 'isin_id_spatial': ['None']},
         'confRaster': {'api': 'GEE', 'masque': [{'gt': 30}], 'outputValue': 'continue', 'defaultValue': 0.0,
                        'overlayHow': 'identity', 'uri_image': 'UMD/hansen/global_forest_change_2021_v1_9',
                        'windows_height': 2048, 'windows_width': 2048}, 'dataName': 'gfc_treecover2000',
         'epsg': 'EPSG:4326', 'keepList': ['treecover2000'], 'overlayHow': 'identity', 'sourceType': 'Raster'}

{'adaptingDataframe': {'changeType': {}, 'renameMap': {}, 'setValue': None, 'colName': 'id_indicateur', 'value': '??', 'toDrop': []}, 'catalogUri': 'None', 'confDb': {'chunksize': 1000, 'schema': 'surfor', 'strategy': 'append', 'tableName': 'faits_GFC_treecover2000'}, 'confDims': {'isin_id_mesure': [3, 100], 'isin_id_spatial': ['None']}, 'confRaster': {'api': 'GEE', 'masque': [{'gt': 30}], 'outputValue': 'continue', 'defaultValue': 0.0, 'overlayHow': 'identity', 'uri_image': 'UMD/hansen/global_forest_change_2021_v1_9', 'windows_height': 2048, 'windows_width': 2048}, 'dataName': 'gfc_treecover2000', 'epsg': 'EPSG:4326', 'keepList': ['treecover2000'], 'overlayHow': 'identity', 'sourceType': 'Raster'}
